# 1.多线程

In [9]:
#Python的标准库提供了两个模块：_thread和threading，_thread是低级模块，
#threading是高级模块，对_thread进行了封装。绝大多数情况下，我们只需要使用threading这个高级模块。
#启动一个线程就是把一个函数传入并创建thread实例，然后start()
import time,threading
def loop():
    print("Threading is running,current thread name is : %s"  %threading.current_thread().name)
    n = 10
    while n>5:
        print("current thread name is :%s" %threading.current_thread().name)
        print(n,"------->",n)
        n = n-1
    print("Thread is end,current therad name is :%s "  %threading.current_thread().name)
        
print(" %s  run begin ....."  %threading.current_thread().name)
t = threading.Thread(target=loop,name="loop_num")
t.start()
t.join()
print("%s  run  begin ....."  %threading.current_thread().name)




 MainThread  run begin .....
Threading is running,current thread name is : loop_num
current thread name is :loop_num
10 -------> 10
current thread name is :loop_num
9 -------> 9
current thread name is :loop_num
8 -------> 8
current thread name is :loop_num
7 -------> 7
current thread name is :loop_num
6 -------> 6
Thread is end,current therad name is :loop_num 
MainThread  run  begin .....


# 2.线程锁

In [24]:
#多进程：多个进程之中，同一个变量，各有一份拷贝在自己的进程中，互不影响。
#多线程：各个线程共同拥有所以变量，任何一个线程都可以任何一个变量。
#多线程共享最大的风险在于多个线程同时修改同一个变量，把内容给改乱了。

#例子：一个变量是如何被改乱的？
import time,threading
balance = 0
def change_balance(n):
    #print("thread name is :%s" %threading.current_thread().name)
    global balance
    balance = balance  + n
    balance = balance  - n
    #print("youe balace is : %s " %balance)
#change_balance(5)

def start_thread(n):
    for i in range(10000000):
        change_balance(n)
#start_thread()
#craete thread instance

t1 = threading.Thread(target=start_thread,args=(5,))
t2 = threading.Thread(target=start_thread,args=(8,)) 
t1.start()
t2.start()
t1.join()
t2.join()
print("your balance is :%s" %balance)
#执行次数足够多的情况下，balance就不等于0

your balance is :24


In [26]:
#如果要确保balance不被改坏，就要给change_balance函数加上锁，当线程执行到change_balance时，我们说该线程获得了锁
#因此其它线程就不能同时执行change_balance()，创建一个锁就是通过threading.Lock()来实现

#改进版的change_balance() 如下：
import time,threading
balance = 0
lock = threading.Lock()
def change_balance(n):
    #print("thread name is :%s" %threading.current_thread().name)
    global balance
    balance = balance  + n
    balance = balance  - n
    #print("youe balace is : %s " %balance)
#change_balance(5)

def start_thread(n):
    for i in range(10000000):
        lock.acquire()
        try:
            change_balance(n)
        finally:
            lock.release()
#start_thread()
#craete thread instance

t1 = threading.Thread(target=start_thread,args=(5,))
t2 = threading.Thread(target=start_thread,args=(8,)) 
t1.start()
t2.start()
t1.join()
t2.join()
print("your balance is :%s" %balance)


your balance is :0


# 3.多核CPU

In [ ]:
import threading, multiprocessing
print(multiprocessing.cpu_count())

def loop():
    n = 2
    while True:
        n = n*n
for i in range(multiprocessing.cpu_count()):
    t = threading.Thread(target=loop)
    t.start()

4


In [ ]:
#ython解释器由于设计时有GIL全局锁，导致了多线程无法利用多核。多线程的并发在Python中就是一个美丽的梦。
#因为Python的线程虽然是真正的线程，但解释器执行代码时，有一个GIL锁：Global Interpreter Lock，
#任何Python线程执行前，必须先获得GIL锁，然后，每执行100条字节码，解释器就自动释放GIL锁，让别的线程有机会执行。这个GIL全局锁实际上把所有线程的执行代码都给上了锁，
#所以，多线程在Python中只能交替执行，即使100个线程跑在100核CPU上，也只能用到1个核。